# Proyek UTS: Sistem Temu Kembali Informasi (STKI)
## Mini Search Engine (Boolean & Vector Space Model)

 Proyek ini adalah implementasi dari sistem temu kembali informasi (STKI) mini sebagai pemenuhan Ujian Tengah Semester (UTS) Ganjil 2025/2026. Sistem ini dibangun menggunakan Python, mampu mengindeks 5 dokumen berita, dan mendukung dua model pencarian: **Boolean Retrieval** dan **Vector Space Model (VSM)** dengan perankingan TF-IDF.

Aplikasi ini juga di-deploy menggunakan Streamlit Community Cloud dan dapat diakses secara publik.

**Tautan Aplikasi Streamlit:** `https://uts-stki-14119.streamlit.app`

---

## 🧑‍🎓 Informasi Mahasiswa

* **Nama:** `Rizka Nugraha`
* **NIM:** `A11.2022.14119`
* **Mata Kuliah:** Sistem Temu Kembali Informasi (A11.4703)
* **Dosen:** Abu Salam, M.Kom
* **Universitas:** Universitas Dian Nuswantoro

---

Notebook Google Colab ini berfungsi sebagai script eksekusi utama untuk proyek UTS STKI. Notebook ini akan menjalankan pengujian dan evaluasi untuk setiap modul yang diminta dalam soal, yaitu:
* **Soal 02:** Document Preprocessing
* **Soal 03:** Boolean Retrieval Model
* **Soal 04:** Vector Space Model (VSM)
* **Soal 05:** Perbandingan Term Weighting & Evaluasi

---



In [123]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [124]:
%cd /content/drive/MyDrive/STKI/UTS

/content/drive/MyDrive/STKI/UTS


In [125]:
%cd /content/drive/MyDrive/STKI/UTS/notebooks

/content/drive/MyDrive/STKI/UTS/notebooks


**Import library/package**

In [126]:
import os
import nltk
import pandas as pd
from pyngrok import ngrok
import shutil

## Persiapan Data Korpus & Arsitektur

Sebelum memulai, kita perlu menyiapkan dua hal:
1.  **Menulis Korpus:** Menulis 5 dokumen berita (yang disimpan sebagai string) ke dalam file `../data/raw/beritaX.txt`. Ini adalah sumber data mentah untuk sistem kita.
2.  **Membuat Package `src`:** Membuat file `../src/__init__.py` yang kosong. Ini memberi tahu Python bahwa folder `src` adalah sebuah *package*, yang memungkinkan kita mengimpor file `.py` di dalamnya (misal: `from src.preprocess import ...`).

In [127]:

print("\n--- Menulis 5 file berita ke 'data/raw'... ---")
berita_corpus = {
    "berita1.txt": """Wilayah Kamu Sudah 'Bebas' COVID-19? Cek 34 Kab/Kota Zona Hijau Terbaru
Jakarta - Pemerintah rencananya bakal menerapkan Pemberlakuan Pembatasan Kegiatan Masyarakat (PPKM) level 3 terhitung 24 Desember 2021 hingga 2 Januari 2021. Namun, Kementerian Kesehatan RI memastikan kebijakan PPKM level 3 ini masih dalam tahap kajian.
Menurut Direktur Pencegahan dan Pengendalian Penyakit Menular Langsung (P2PML) Kemenkes RI dr Siti Nadia Tarmizi, PPKM level 3 bakal diterapkan jika kasus COVID-19 naik signifikan. Hal ini umumnya dipicu dengan peningkatan mobilitas dan pelonggaran protokol kesehatan.
https://health.detik.com/berita-detikhealth/d-5816690/wilayah-kamu-sudah-bebas-covid-19-cek-34-kabkota-zona-hijau-terbaru""",

    "berita2.txt": """Vaksin COVID-19 Bakal Rutin Setiap Tahun? Tergantung, Ini Penjelasannya
Jakarta - Pemberian booster atau dosis ketiga vaksin COVID-19 di Indonesia direncanakan bakal berlangsung Januari 2022. Lantas adakah kemungkinan vaksinasi COVID-19 bakal harus dilakukan setiap tahun seperti vaksinasi influenza?
Ketua Satgas COVID-19 Ikatan Dokter Indonesia (IDI) Prof Zubairi Djoerban menjelaskan hingga kini belum ada kepastian terkait hal tersebut. Menurutnya masih ada kemungkinan vaksin COVID-19 harus diberikan setiap tahun, ada juga kemungkinan cukup booster diberikan sekali kemudian vaksinasi COVID-19 tidak diperlukan lagi.
https://health.detik.com/berita-detikhealth/d-5816582/vaksin-covid-19-bakal-rutin-setiap-tahun-tergantung-ini-penjelasannya""",

    "berita3.txt": """RI Mulai Suntikkan Booster di 2022, Masihkah Ampuh Lawan Varian Delta Cs?
Jakarta - Pakar mengakui vaksin-vaksin yang sudah digunakan untuk dosis 1-2 memang mengalami penurunan efektivitas terhadap varian baru Corona seperti varian Delta. Mengingat booster atau dosis ketiga vaksin COVID-19 di Indonesia disebut bakal dimulai awal 2022, apakah jenis vaksin yang digunakan bakal mengikuti strain virus terbaru?
Menjawab pertanyaan tersebut, Ketua Satgas COVID-19 Ikatan Dokter Indonesia (IDI) Prof Zubairi Djoerban kembali menyinggung riset yang sudah berlangsung terkait efektivitas vaksin COVID-19 dosis 1 dan 2. Ia menyebut berdasarkan riset sejauh ini, efektivitas vaksin COVID-19 Pfizer dan Moderna terbukti menurun dalam melawan varian Delta.
https://health.detik.com/berita-detikhealth/d-5816534/ri-mulai-suntikkan-booster-di-2022-masihkah-ampuh-lawan-varian-delta-cs""",

    "berita4.txt": """Alert! Kasus Varian Delta COVID-19 di DKI Meningkat
Jakarta - Data terbaru dari Balitbangkes Kemenkes RI per 13 November, menunjukkan adanya penambahan varian Delta. Penambahan tersebut terjadi di Jawa Barat ada 165 kasus, DKI Jakarta 90 kasus, dan Sulawesi Utara 86 kasus.
Dalam satu bulan terakhir, Balitbangkes menyebutkan DKI Jakarta mengalami peningkatan kasus varian Delta yang signifikan. Sementara, pada varian baru seperti varian Alpha, varian Delta, hingga Beta terbanyak di Indonesia berasal dari DKI Jakarta, dengan total 1.327 kasus .
https://health.detik.com/berita-detikhealth/d-5812940/alert-kasus-varian-delta-covid-19-di-dki-meningkat""",

    "berita5.txt": """Corona di AS Mendadak Naik Lagi Usai Serangan Delta Sempat Mereda
Jakarta - Kasus COVID-19 kembali naik di sejumlah wilayah Amerika Serikat (AS). Padahal diketahui, COVID-19 sempat tercatat stabil pasca serangan varian Delta musim panas ini. Ada apa?
Hal tersebut disampaikan oleh kepala penasihat medis Gedung Putih Dr. Anthony Fauci, Senin (15/11/2021). Diketahui, kasus nasional turun 57 persen minggu lalu dari puncak gelombang varian Delta pada musim panas. Namun jumlah pasien COVID-19 di area Barat Tengah dan Timur laut kini naik mendadak.
https://health.detik.com/berita-detikhealth/d-5813949/corona-di-as-mendadak-naik-lagi-usai-serangan-delta-sempat-mereda"""
}

for filename, content in berita_corpus.items():
    with open(os.path.join("../data/raw", filename), "w", encoding="utf-8") as f:
        f.write(content)

print(f"{len(berita_corpus)} file berita berhasil ditulis.")


--- Menulis 5 file berita ke 'data/raw'... ---
5 file berita berhasil ditulis.


In [128]:
import os

# Path ke file __init__.py di dalam folder src
init_file_path = "../src/__init__.py"

# Cek jika file belum ada, lalu buat
if not os.path.exists(init_file_path):
    with open(init_file_path, "w") as f:
        pass # Buat file kosong
    print(f"File '{init_file_path}' berhasil dibuat.")
    print("Folder 'src' sekarang sudah menjadi package Python.")
else:
    print(f"File '{init_file_path}' sudah ada. Tidak perlu dibuat ulang.")

print("\nSilakan coba jalankan ulang sel yang menjalankan aplikasi Streamlit Anda.")

File '../src/__init__.py' sudah ada. Tidak perlu dibuat ulang.

Silakan coba jalankan ulang sel yang menjalankan aplikasi Streamlit Anda.


## Download Dependensi NLTK

Sel-sel berikutnya mengunduh data yang diperlukan oleh library NLTK:
* `punkt`: Diperlukan untuk *tokenisasi* (memecah teks menjadi kata).
* `stopwords`: Berisi daftar kata umum Bahasa Indonesia (seperti 'dan', 'di', 'yang') untuk dihapus.
* `punkt_tab`: Data tambahan untuk tokenisasi.

In [129]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [130]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [131]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Eksekusi Soal 02: Document Preprocessing

Sel ini menjalankan implementasi **Soal 02**.

Prosesnya adalah sebagai berikut:
1.  Mengimpor fungsi `preprocess` dari `../src/preprocess.py`.
2.  Membaca setiap file dari `../data/raw/`.
3.  Menjalankan pipeline preprocessing (case-folding, tokenisasi, stopword removal, stemming).
4.  Menyimpan hasilnya ke `../data/processed/`.

Sebagai bukti, sel ini akan mencetak perbandingan **"BEFORE"** (teks mentah) dan **"AFTER"** (teks yang sudah diproses) untuk dua dokumen pertama.

In [132]:

print("--- Menjalankan Soal 02: Document Preprocessing... ---")
import sys
import os # Import os untuk path handling

# Tambahkan path saat ini (yaitu folder 'UTS') ke sys.path
# Ini agar Python bisa menemukan folder 'src'
sys.path.append(os.path.abspath('../'))

# --- PATH DIPERBAIKI ---
# Karena Anda sudah di dalam 'UTS', 'UTS/' dihapus dari awal path
raw_dir = "../data/raw"
processed_dir = "../data/processed"
# ---------------------

# Pastikan folder 'src' ada
if not os.path.exists('../src'):
     print("ERROR: Folder 'src' tidak ditemukan. Pastikan Anda berada di folder 'UTS'.")
else:
    from src.preprocess import preprocess

    # Ambil 2 sampel dokumen untuk before/after
    #
    sample_files = sorted(os.listdir(raw_dir))[:2]
    print("\n--- Menampilkan Before/After untuk 2 Sampel Dokumen ---")

    processed_log = []

    for doc_id in sorted(os.listdir(raw_dir)):
        if not doc_id.endswith('.txt'): continue

        with open(os.path.join(raw_dir, doc_id), 'r', encoding='utf-8') as f_in:
            raw_content = f_in.read()

        processed_content = preprocess(raw_content)

        # Simpan keluaran akhir ke data/processed/*.txt
        #
        with open(os.path.join(processed_dir, doc_id), 'w', encoding='utf-8') as f_out:
            f_out.write(processed_content)

        log_entry = f"Dokumen '{doc_id}': Selesai diproses."
        processed_log.append(log_entry)

        # Tampilkan 2 sampel
        if doc_id in sample_files:
            print(f"\n[DOKUMEN: {doc_id}]")
            print("--- BEFORE ---")
            print(raw_content[:150] + "...")
            print("\n--- AFTER ---")
            print(processed_content[:150] + "...")

    print("\n--- Log Ringkas Proses ---")
    # (log ringkas)
    for log in processed_log:
        print(log)
    print("\nPreprocessing selesai. Hasil disimpan di 'data/processed/'.")

--- Menjalankan Soal 02: Document Preprocessing... ---

--- Menampilkan Before/After untuk 2 Sampel Dokumen ---

[DOKUMEN: berita1.txt]
--- BEFORE ---
Wilayah Kamu Sudah 'Bebas' COVID-19? Cek 34 Kab/Kota Zona Hijau Terbaru
Jakarta - Pemerintah rencananya bakal menerapkan Pemberlakuan Pembatasan Kegia...

--- AFTER ---
wilayah bebas covid cek kabkota zona hijau baru jakarta perintah rencana terap laku batas giat masyarakat ppkm level hitung desember januari menteri s...

--- Log Ringkas Proses ---
Dokumen 'berita1.txt': Selesai diproses.
Dokumen 'berita2.txt': Selesai diproses.
Dokumen 'berita3.txt': Selesai diproses.
Dokumen 'berita4.txt': Selesai diproses.
Dokumen 'berita5.txt': Selesai diproses.

Preprocessing selesai. Hasil disimpan di 'data/processed/'.


In [133]:
!cat ../data/raw/berita1.txt

Wilayah Kamu Sudah 'Bebas' COVID-19? Cek 34 Kab/Kota Zona Hijau Terbaru
Jakarta - Pemerintah rencananya bakal menerapkan Pemberlakuan Pembatasan Kegiatan Masyarakat (PPKM) level 3 terhitung 24 Desember 2021 hingga 2 Januari 2021. Namun, Kementerian Kesehatan RI memastikan kebijakan PPKM level 3 ini masih dalam tahap kajian.
Menurut Direktur Pencegahan dan Pengendalian Penyakit Menular Langsung (P2PML) Kemenkes RI dr Siti Nadia Tarmizi, PPKM level 3 bakal diterapkan jika kasus COVID-19 naik signifikan. Hal ini umumnya dipicu dengan peningkatan mobilitas dan pelonggaran protokol kesehatan.
https://health.detik.com/berita-detikhealth/d-5816690/wilayah-kamu-sudah-bebas-covid-19-cek-34-kabkota-zona-hijau-terbaru

In [134]:
!cat ../data/processed/berita1.txt

wilayah bebas covid cek kabkota zona hijau baru jakarta perintah rencana terap laku batas giat masyarakat ppkm level hitung desember januari menteri sehat ri bijak ppkm level tahap kaji direktur cegah kendali sakit tular langsung ppml kemenkes ri dr siti nadia tarmizi ppkm level terap covid signifikan picu tingkat mobilitas longgar protokol sehat

## Eksekusi Soal 03: Boolean Retrieval Model

Sel ini menjalankan implementasi **Soal 03**.

Prosesnya adalah:
1.  Mengimpor kelas `BooleanRetrieval` dari `../src/boolean_ir.py`.
2.  Kelas ini secara otomatis membuat **Inverted Index** dari file di `../data/processed/`.
3.  Menjalankan 3 kueri uji untuk operator `AND`, `OR`, dan `NOT`.
4.  Menjalankan evaluasi wajib (Soal 03) terhadap *gold standard* manual untuk menghitung **Precision, Recall, dan F1-Score**.

In [135]:

print("\n--- Menjalankan Soal 03: Boolean Retrieval Model... ---")
#
from src.boolean_ir import BooleanRetrieval
from src.eval import calculate_precision_recall_f1

processed_dir = "../data/processed"
bool_model = BooleanRetrieval(processed_dir)

# Uji dengan >= 3 query
#
queries_to_test = [
    "vaksin AND delta",
    "ppkm OR jakarta",
    "kasus NOT amerika"
]

print("\n--- Pengujian Query Boolean ---")
query_results_bool = {}
for q in queries_to_test:
    docs, explain = bool_model.process_query(q)
    query_results_bool[q] = docs
    print(f"\nQuery: '{q}'")
    print(f"Explain: {explain}")
    print(f"Hasil: {docs}")

# Uji Wajib (mini truth set)
#
print("\n--- Evaluasi Wajib Boolean (Precision/Recall) ---")
# Buat gold set (ground truth) manual.
# ANDA BISA UBAH INI SESUAI ANALISIS ANDA
gold_standard_bool = {
    "vaksin AND delta": {"berita3.txt"},
    "ppkm OR jakarta": {"berita1.txt", "berita4.txt"},
    "kasus NOT amerika": {"berita1.txt", "berita2.txt", "berita3.txt", "berita4.txt"}
}
print(f"Gold Standard (Asumsi): {gold_standard_bool}")

eval_results = []
for query, relevant_docs in gold_standard_bool.items():
    retrieved_docs = query_results_bool.get(query, [])
    p, r, f1 = calculate_precision_recall_f1(retrieved_docs, relevant_docs)
    eval_results.append({
        "Query": query,
        "Retrieved": len(retrieved_docs),
        "Relevant": len(relevant_docs),
        "Precision": p,
        "Recall": r,
        "F1-Score": f1
    })

df_eval_bool = pd.DataFrame(eval_results)
print("\nHasil Evaluasi Boolean:")
print(df_eval_bool.to_string())


--- Menjalankan Soal 03: Boolean Retrieval Model... ---

--- Pengujian Query Boolean ---

Query: 'vaksin AND delta'
Explain: vaksin (2 dok) AND delta -> 1 dok
Hasil: ['berita3.txt']

Query: 'ppkm OR jakarta'
Explain: ppkm (1 dok) OR jakarta -> 5 dok
Hasil: ['berita1.txt', 'berita2.txt', 'berita3.txt', 'berita4.txt', 'berita5.txt']

Query: 'kasus NOT amerika'
Explain: NOT amerika (4 dok)
Hasil: ['berita1.txt', 'berita2.txt', 'berita3.txt', 'berita4.txt']

--- Evaluasi Wajib Boolean (Precision/Recall) ---
Gold Standard (Asumsi): {'vaksin AND delta': {'berita3.txt'}, 'ppkm OR jakarta': {'berita1.txt', 'berita4.txt'}, 'kasus NOT amerika': {'berita1.txt', 'berita2.txt', 'berita3.txt', 'berita4.txt'}}

Hasil Evaluasi Boolean:
               Query  Retrieved  Relevant  Precision  Recall  F1-Score
0   vaksin AND delta          1         1        1.0     1.0  1.000000
1    ppkm OR jakarta          5         2        0.4     1.0  0.571429
2  kasus NOT amerika          4         4        1.0    

## 6. Eksekusi Soal 04: Vector Space Model (VSM) & Ranking

Sel ini menjalankan implementasi **Soal 04**.

Prosesnya adalah:
1.  Mengimpor kelas `VectorSpaceModel` dari `../src/vsm_ir.py`.
2.  Kelas ini secara otomatis:
    * Memuat data mentah dari `../data/raw/`.
    * Melakukan preprocessing.
    * Membangun **Matriks TF-IDF** (sepotong matriks ditampilkan sebagai bukti).
3.  Menjalankan kueri uji ("varian delta di jakarta") dan menampilkan hasil **Top-3** yang diurutkan berdasarkan skor **Cosine Similarity**.
4. Menjalankan evaluasi wajib (Soal 04) terhadap *gold standard* untuk menghitung **Precision@k** dan **MAP@k (Mean Average Precision)**.

In [136]:

print("\n--- Menjalankan Soal 04: Vector Space Model & Ranking... ---")
#
from src.vsm_ir import VectorSpaceModel
from src.eval import precision_at_k, mean_average_precision

raw_dir = "../data/raw"
vsm_model = VectorSpaceModel(raw_dir)

# Tampilkan sebagian matriks TF-IDF
print("\n--- Menampilkan 5 Term Pertama dari Matriks TF-IDF ---")
df_tfidf = vsm_model.get_tfidf_matrix()
print(df_tfidf.iloc[:, :5].to_string())

# Uji query
print("\n--- Pengujian Query VSM (Top-3) ---")
# (ambil top-k, k=3)
query_vsm = "varian delta di jakarta"
k_vsm = 3
vsm_search_results = vsm_model.search(query_vsm, k=k_vsm)

print(f"Query: '{query_vsm}'")
print(f"Top-{k_vsm} Hasil:")
for res in vsm_search_results:
    print(f"  - [{res['doc_id']}] Score: {res['score']:.4f} | Snippet: {res['snippet']}")

# Uji Wajib: bandingkan dengan gold set
#
print("\n--- Evaluasi Wajib VSM (P@k & MAP@k) ---")
# Kita pakai query yang sama dengan boolean untuk perbandingan
gold_standard_vsm = {
    "vaksin AND delta": {"berita2.txt", "berita3.txt"}, # VSM mungkin menangkap 'vaksin' di berita2
    "ppkm OR jakarta": {"berita1.txt", "berita4.txt"},
    "kasus NOT amerika": {"berita1.txt", "berita4.txt"} # VSM sulit menangani 'NOT'
}

# Dapatkan hasil VSM untuk setiap query di gold standard
query_results_vsm = {}
for query in gold_standard_vsm.keys():
    query_results_vsm[query] = vsm_model.search(query, k=5) # Ambil K=5

# Hitung P@k (P@3) dan MAP@k (MAP@5)
#
eval_vsm_results = []
map_k = 5
p_k = 3

for query, relevant_docs in gold_standard_vsm.items():
    retrieved_docs_data = query_results_vsm.get(query, [])
    retrieved_doc_ids = [r['doc_id'] for r in retrieved_docs_data]

    p_at_k = precision_at_k(retrieved_doc_ids, relevant_docs, k=p_k)
    eval_vsm_results.append({
        "Query": query,
        f"P@{p_k}": p_at_k
    })

map_score = mean_average_precision(query_results_vsm, gold_standard_vsm, k=map_k)

print(f"Gold Standard (Asumsi): {gold_standard_vsm}")
df_eval_vsm = pd.DataFrame(eval_vsm_results)
print(f"\nHasil Evaluasi VSM (Precision@{p_k}):")
print(df_eval_vsm.to_string())
print(f"\nSKOR KESELURUHAN: MAP@{map_k} = {map_score:.4f}")


--- Menjalankan Soal 04: Vector Space Model & Ranking... ---

--- Menampilkan 5 Term Pertama dari Matriks TF-IDF ---
                  ada       aku     alami     alert     alpha
berita1.txt  0.000000  0.000000  0.000000  0.000000  0.000000
berita2.txt  0.148874  0.000000  0.000000  0.000000  0.000000
berita3.txt  0.000000  0.111765  0.090171  0.000000  0.000000
berita4.txt  0.000000  0.000000  0.098029  0.121504  0.121504
berita5.txt  0.000000  0.000000  0.000000  0.000000  0.000000

--- Pengujian Query VSM (Top-3) ---
Query: 'varian delta di jakarta'
Top-3 Hasil:
  - [berita4.txt] Score: 0.6181 | Snippet: Alert! Kasus Varian Delta COVID-19 di DKI Meningkat Jakarta - Data terbaru dari Balitbangkes Kemenkes RI per 13 November...
  - [berita3.txt] Score: 0.3549 | Snippet: RI Mulai Suntikkan Booster di 2022, Masihkah Ampuh Lawan Varian Delta Cs? Jakarta - Pakar mengakui vaksin-vaksin yang su...
  - [berita5.txt] Score: 0.3159 | Snippet: Corona di AS Mendadak Naik Lagi Usai Serangan Delt

## Eksekusi Soal 05: Perbandingan Term Weighting

Sel ini menjalankan implementasi inti dari **Soal 05 (Langkah 1 & 4)**, yaitu membandingkan dua skema *term weighting*.

Prosesnya adalah:
1.  **Memuat Ulang Modul:** `importlib.reload` digunakan untuk memastikan notebook ini menggunakan versi terbaru `vsm_ir.py` (yang telah diupdate untuk menerima opsi `sublinear_tf`).
2.  **Inisialisasi Model:** Membuat dua instance `VectorSpaceModel`:
    * `vsm_model_default`: Menggunakan TF-IDF standar (`sublinear_tf=False`).
    * `vsm_model_sublinear`: Menggunakan TF-IDF Sublinear (`sublinear_tf=True`).
3.  **Evaluasi:** Menjalankan *gold standard* yang sama pada kedua model.
4.  **Laporan:** Mencetak tabel perbandingan skor **MAP@5** dari kedua skema, diikuti dengan analisis singkat.

In [137]:

print("\n--- [Eksekusi Soal 05]: Perbandingan Term Weighting (Langkah 1 & 4) ---")

import importlib
import pandas as pd
import sys


# Muat ulang modul yang baru saja kita ubah
import src.vsm_ir
import src.eval
importlib.reload(src.vsm_ir)
importlib.reload(src.eval)

from src.vsm_ir import VectorSpaceModel
from src.eval import mean_average_precision

raw_dir = "../data/raw"

# 1. Inisialisasi model dengan 2 skema berbeda
print("Membuat model VSM (Default TF-IDF)...")
vsm_model_default = VectorSpaceModel(raw_dir, sublinear_tf=False)

print("Membuat model VSM (Sublinear TF-IDF)...")
vsm_model_sublinear = VectorSpaceModel(raw_dir, sublinear_tf=True)

# 2. Siapkan Gold Standard (sama seperti Soal 04)
#
gold_standard_vsm = {
    "vaksin AND delta": {"berita2.txt", "berita3.txt"},
    "ppkm OR jakarta": {"berita1.txt", "berita4.txt"},
    "kasus NOT amerika": {"berita1.txt", "berita4.txt"}
}
map_k = 5

# 3. Evaluasi Skema 1 (Default)
query_results_default = {}
for query in gold_standard_vsm.keys():
    query_results_default[query] = vsm_model_default.search(query, k=map_k)
map_default = mean_average_precision(query_results_default, gold_standard_vsm, k=map_k)

# 4. Evaluasi Skema 2 (Sublinear)
query_results_sublinear = {}
for query in gold_standard_vsm.keys():
    query_results_sublinear[query] = vsm_model_sublinear.search(query, k=map_k)
map_sublinear = mean_average_precision(query_results_sublinear, gold_standard_vsm, k=map_k)

# 5. Laporkan pengaruhnya terhadap metrik
#
print("\n--- Laporan Perbandingan Metrik (MAP@5) ---")
# (Tabel perbandingan, bukan grafik)
comparison_data = {
    "Skema Weighting": ["Default TF-IDF", "Sublinear TF-IDF"],
    f"MAP@{map_k}": [map_default, map_sublinear]
}
df_comparison = pd.DataFrame(comparison_data).set_index("Skema Weighting")
print(df_comparison.to_string())

print("\n--- Analisis Singkat (Wajib untuk Laporan) ---")
print(f"Penggunaan Sublinear TF-IDF (MAP={map_sublinear:.4f}) menghasilkan skor yang "
      f"{'lebih tinggi' if map_sublinear > map_default else 'lebih rendah atau sama'}"
      f" dibandingkan Default TF-IDF (MAP={map_default:.4f}) pada gold set ini.")
print("Sublinear TF mengganti 'tf' mentah dengan '1 + log(tf)',")
print("yang berguna untuk menangani perbedaan frekuensi kata yang ekstrem.")


--- [Eksekusi Soal 05]: Perbandingan Term Weighting (Langkah 1 & 4) ---
Membuat model VSM (Default TF-IDF)...
Membuat model VSM (Sublinear TF-IDF)...

--- Laporan Perbandingan Metrik (MAP@5) ---
                     MAP@5
Skema Weighting           
Default TF-IDF    0.666667
Sublinear TF-IDF  0.666667

--- Analisis Singkat (Wajib untuk Laporan) ---
Penggunaan Sublinear TF-IDF (MAP=0.6667) menghasilkan skor yang lebih rendah atau sama dibandingkan Default TF-IDF (MAP=0.6667) pada gold set ini.
Sublinear TF mengganti 'tf' mentah dengan '1 + log(tf)',
yang berguna untuk menangani perbedaan frekuensi kata yang ekstrem.
